In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import keras
from keras.layers import Dense, Activation
from keras.models import Sequential
import tensorflow as tf

In [2]:
import gym

env = gym.make("FrozenLake8x8-v0")
env.reset()

#plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
print('observation space:', env.observation_space)
print('action space:', env.action_space)
print("taking action")
new_obs, reward, is_done, _ = env.step(1)

print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)


Observation space: Discrete(64)
Action space: Discrete(4)
observation space: Discrete(64)
action space: Discrete(4)
taking action
new observation code: 1
reward: 0.0
is game over?: False


In [3]:
n_states = env.observation_space.n
n_actions = env.action_space.n
print("Number of States: ", n_states)
print("Number of Actions: ", n_actions)

Number of States:  64
Number of Actions:  4


In [4]:
#Input policy to be evaluated

policy = np.ones((n_states , n_actions))/n_actions
print("Shape of Policy Matrix" , policy.shape)

#Initialize array V(s) = 0

V = np.zeros(n_states)
discount_factor = 1.0
theta = 0.00001

Shape of Policy Matrix (64, 4)


In [6]:
env.P[3][3]

[(0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 3, 0.0, False),
 (0.3333333333333333, 2, 0.0, False)]

In [43]:
#Helper function which calculates the value for all actions
def one_step_lookahead(state , V , discount_factor = 1.0):
    v = np.zeros(n_actions)
    for a in range(n_actions):
        for prob, next_state, reward, done  in env.P[state][a]:
            v[a] = prob*(reward+discount_factor*V[next_state])
    return v
""""
Formula:
      V = sum(prob(s', r|s,a)) * (reward + gamma*V(s')) 

### Policy Evaluation

In [7]:
def policy_evaluation(policy, env, discount_factor = 1.0 , theta = 0.00001 ):
    #Repeat:
    while True:
        delta = 0.0        #delta <- 0
    
        for s in range(n_states):   #For each state:
            v = 0
            for a , policy_prob in enumerate(policy[s]):                            #Expression---
                for prob, next_state, reward, done  in env.P[s][a]:
                    v += policy_prob * prob *(reward + discount_factor*V[next_state])     # V(s) = sum(policy(a|s) * sum(prob(s', r|s,a)) * (reward + gamma*V(s')) 
            delta = max(delta , abs(v - V[s]))            #delta <- max(delta , |v - V[s]|)
            V[s] = v

        if(delta<theta):                    #terminate on delta < theta (theta is a small positive number)
            break
    return V
        # Output V ~= vpi

In [42]:
vpi = policy_evaluation(policy , env) 
print("Value: " , vpi)

Value:  [2.84835099e-03 3.14099803e-03 3.83307468e-03 5.31436453e-03
 8.61087838e-03 1.30662629e-02 1.81048038e-02 2.11866439e-02
 2.53651262e-03 2.73445587e-03 3.03991188e-03 3.49704871e-03
 7.45066365e-03 1.24821542e-02 2.00606634e-02 2.42676012e-02
 2.01868693e-03 2.01804649e-03 1.78767009e-03 0.00000000e+00
 6.37212215e-03 9.93466621e-03 2.34317315e-02 3.15552911e-02
 1.49608503e-03 1.52925294e-03 2.32221776e-03 4.66264019e-03
 1.16654746e-02 0.00000000e+00 2.88049879e-02 4.69664404e-02
 9.36955988e-04 7.67873022e-04 7.74072587e-04 0.00000000e+00
 2.39616613e-02 3.78061768e-02 3.94484063e-02 8.05389966e-02
 5.45444953e-04 0.00000000e+00 0.00000000e+00 1.27795527e-02
 3.40788072e-02 8.94568690e-02 0.00000000e+00 1.55202126e-01
 6.98915628e-04 0.00000000e+00 1.70394036e-03 4.25985091e-03
 0.00000000e+00 1.96485623e-01 0.00000000e+00 3.85067375e-01
 8.51930150e-04 8.51948220e-04 8.51970181e-04 0.00000000e+00
 2.50000000e-01 5.00000000e-01 7.50000000e-01 0.00000000e+00]


### Policy Iteration

In [40]:
def policy_iteration(env , discount_factor=1.0):
    #1. Initialization
    policy = np.ones((n_states , n_actions))/n_actions

    while True:
        #2. Policy Evaluation
        V = policy_evaluation(policy , env) 
    
        #3. Policy Improvement
        policy_stable = True
        
        for s in range(n_states):  #for each s in States
            old_action = np.argmax(policy[s])            #old_action <- policy[s]
            best_action = np.argmax(one_step_lookahead(s , V))  #policy[s]<- sum(prob(s', r|s,a)) * (reward + gamma*V(s')) 
            if old_action != best_action:           #if old-action != policy(s), then policy-stable -> false
                policy_stable = False 
            policy[s] = np.eye(n_actions)[best_action]   #(Making the index of policy for the best action as 1)
        if policy_stable:         #if policy-stable, then stop and return V  and policy   
            return policy, V

In [41]:
policy , V = policy_iteration(env)
print("Policy: ", policy)
print("Value: " , V)

Policy:  [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
Value:  [2.85609659e-03 3.1464

### Value Iteration

In [34]:
def value_iteration(env , discount_factor = 1.0 , theta = 0.00001 ):
    V = np.zeros(n_states)     #Initialize array V arbitrarily
    
    #Repeat
    while True:    
        delta = 0.0        #delta <- 0
        for s in range(n_states):   #For each s in state:
            v = V[s]               # v <- V(s)
            V[s] = max(one_step_lookahead(s, V , discount_factor = discount_factor))  # V(s) = max(sum(prob(s', r|s,a)) * (reward + gamma*V(s'))) 
            delta = max(delta , abs(v - V[s]))            #delta <- max(delta , |v - V[s]|)
            
        if(delta<theta):                    #terminate on delta < theta (theta is a small positive number)
            break

    #Output a deterministic policy pi such that:
    policy = np.zeros((n_states, n_actions))

    for s in range(n_states):
        best_action = np.argmax(one_step_lookahead(s , V))
        policy[s] = np.eye(n_actions)[best_action]
    
    return policy, V

In [38]:
policy , V = value_iteration(env)
print("Policy: ", policy)
print("Value: " , V)

Policy:  [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
Value:  [0.00000000e+00 0.0000

Policy:  [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
Value:  [0.00000000e+00 0.0000